# Cinetel Data Cleaning

Cinetel data scraped daily from their boxoffice published everyday at this [link](https://www.cinetel.it/pages/boxoffice.php?edperiodo=aWVyaQ==).

## Reading In Files

In [34]:
import pandas as pd

In [35]:
df1 = pd.read_csv(r"E:\data_analysis_python\cinetel\cinetel_.csv")
df1.head()

,Pos.,Titolo,Prima Progr.,Nazione,Distribuzione,Incasso,Presenze,Incasso al 16/11/2023,Presenze al 16/11/2023,2023-11-17 15:31:50.351708
0,1,C'E' ANCORA DOMANI,'2023-10-26'26/10/2023,ITA,VISION DISTRIBUTION,449991.06,66700,15231636.85,2202169,2023-11-17 15:31:50.351708
1,2,HUNGER GAMES - LA BALLATA DELL'USIGNOLO E DEL ...,'2023-11-15'15/11/2023,USA,MEDUSA FILM S.P.A.,238466.31,32469,584902.45,79311,2023-11-17 15:31:50.351708
2,3,THE MARVELS,'2023-11-08'08/11/2023,USA,WALT DISNEY S.M.P. ITALIA,60914.35,8628,2260142.44,305291,2023-11-17 15:31:50.351708
3,4,THANKSGIVING,'2023-11-16'16/11/2023,USA,EAGLE PICTURES S.P.A.,41232.57,5709,41232.57,5709,2023-11-17 15:31:50.351708
4,5,THE OLD OAK,'2023-11-16'16/11/2023,FRA,LUCKY RED DISTRIB.,36605.89,5879,56540.79,8609,2023-11-17 15:31:50.351708


---

In [36]:
df1 = pd.DataFrame(df1)

## Cleaning the dataframe

To clean the dataframe I decided to drop duplicate values and columns that aren't relevant to my analysis, such as `df1['Incasso al 16/11/2023']` and `df1['Presenze al 16/11/2023]`. I then decided to rename the existing columns using the snake_case rule in order to make the analysis easier.

In [37]:
df1 = df1.drop_duplicates()

In [38]:
# df1 = df1.drop(columns= ['Incasso al 16/11/2023', 'Presenze al 16/11/2023'])

# at first i thought these columns weren't relevant
# making visualizations, i realized i need the data to have accurate insights on movies that started their screenings before i started to scrape the data

In [39]:
df1 = df1.rename(columns = {'Pos.':'daily_rank', 'Titolo' : 'title', 'Prima Progr.' : 'first_screening_date', 'Nazione' : 'nation', 'Distribuzione' : 'distribution', 'Incasso' : 'daily_takings', 'Presenze' : 'daily_attendance','Incasso al 16/11/2023' : 'total_takings',  'Presenze al 16/11/2023' : 'total_attendance', '2023-11-17 15:31:50.351708' : 'date_pulled'})

In [40]:
# dropping the extra 'header row' that is imported everyday when the .csv file gets updated
df1 = df1[df1['daily_rank'].str.contains('Pos.')==False]

In [41]:
# making the 'title' and 'distribution' entries consistent by appling the title format
df1['title'] = df1['title'].apply(lambda x: x.title())
df1['distribution'] = df1['distribution'].apply(lambda x: x.title())

In [42]:
# cleaning the 'first_screening_date' entries
df1['first_screening_date'] = df1['first_screening_date'].str[12:]

In [43]:
# cleaning the column showing the date the data was pulled from the site
df1['date_pulled'] = pd.to_datetime(df1['date_pulled'])
df1['date_pulled'] = pd.to_datetime(df1['date_pulled']).dt.date

#extra tries:
#df1['date_pulled'] = df1['date_pulled'].str[:10]
#df1['date_pulled'] = df1['date_pulled'].str.replace('[^a-zA-Z0-9]', '', regex = True)
#df1['date_pulled'] = df1['date_pulled'].apply(lambda x: str(x))
#df1['date_pulled'] = df1['date_pulled'].apply(lambda x: x[0:2] + '/' + x[2:4] + '/' + x[4:8])

In [44]:
from datetime import timedelta, datetime

In [45]:
df1['first_screening_date']

0      26/10/2023
1      15/11/2023
2      08/11/2023
3      16/11/2023
          ...    
259    12/11/2023
260    30/11/2023
261    30/11/2023
262    30/11/2023
Name: first_screening_date, Length: 240, dtype: object

In [46]:
# formatting
df1['first_screening_date'] = pd.to_datetime(df1['first_screening_date'], errors='coerce')
df1['first_screening_date'] = pd.to_datetime(df1['first_screening_date']).dt.date

C:\Users\Franci\AppData\Local\Temp\ipykernel_27896\2741308956.py:2: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df1['first_screening_date'] = pd.to_datetime(df1['first_screening_date'], errors='coerce')


In [47]:
df1['daily_takings'] = df1['daily_takings'].astype(float)
df1['daily_attendance'] = df1['daily_attendance'].astype(int)
df1['daily_rank'] = df1['daily_rank'].astype(int)
df1['total_takings'] = df1['total_takings'].astype(float)
df1['total_attendance'] = df1['total_attendance'].astype(int)

---

## Data manipulation

I decided to create new columns to get information I may need later on during the analysis.

In [48]:
# calculating the number of days of screenings for each movie 
df1['screening_days'] = datetime.today().date() - df1['first_screening_date']
df1['screening_days'] = df1['screening_days'].map(lambda x: str(x)[:-14])

In [49]:
df1['screening_days']

0      45
1      25
2      32
3      24
       ..
259    28
260    10
261    10
262    10
Name: screening_days, Length: 240, dtype: object

In [50]:
#df1['screening_days'] = df1['screening_days'].str.strip()
df1['screening_days'] = df1['screening_days'].replace('', '0')
df1['screening_days'] = df1['screening_days'].astype(int) + 1

In [51]:
# creating a column to show the date the 'daily_takings' and 'daily_attendace' refer to
df1['date'] = df1['date_pulled'] - timedelta(days=1)
df1['date'] = pd.to_datetime(df1['date'])

In [52]:
# creating a column showing the day of the week. it'll be useful for further analysis
df1['day_of_week'] = df1['date'].dt.day_name()

In [53]:
df1['avg_ticket_price'] = df1['daily_takings'] / df1['daily_attendance']

In [54]:
pd.set_option('display.max_rows', 8)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
df1

,daily_rank,title,first_screening_date,nation,distribution,daily_takings,daily_attendance,total_takings,total_attendance,date_pulled,screening_days,date,day_of_week,avg_ticket_price
0,1,C'E' Ancora Domani,2023-10-26,ITA,Vision Distribution,449991.06,66700,15231636.85,2202169,2023-11-17,46,2023-11-16,Thursday,6.75
1,2,Hunger Games - La Ballata Dell'Usignolo E Del ...,2023-11-15,USA,Medusa Film S.P.A.,238466.31,32469,584902.45,79311,2023-11-17,26,2023-11-16,Thursday,7.34
2,3,The Marvels,2023-11-08,USA,Walt Disney S.M.P. Italia,60914.35,8628,2260142.44,305291,2023-11-17,33,2023-11-16,Thursday,7.06
3,4,Thanksgiving,2023-11-16,USA,Eagle Pictures S.P.A.,41232.57,5709,41232.57,5709,2023-11-17,25,2023-11-16,Thursday,7.22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,7,Improvvisamente A Natale Mi Sposo,2023-11-12,ITA,Notorious Pict. S.P.A.,93194.54,12820,235234.19,32966,2023-12-10,29,2023-12-09,Saturday,7.27
260,8,Diabolik - Chi Sei?,2023-11-30,ITA,01 Distribution,68426.69,9305,581168.53,84422,2023-12-10,11,2023-12-09,Saturday,7.35
261,9,La Guerra Dei Nonni,2023-11-30,ITA,Medusa Film S.P.A.,66173.90,9586,528769.04,78479,2023-12-10,11,2023-12-09,Saturday,6.90
262,10,Home Education - Le Regole Del Male,2023-11-30,ITA,Warner Bros Italia S.P.A.,48297.92,5831,376182.74,48085,2023-12-10,11,2023-12-09,Saturday,8.28


---

## Export as .csv

In [55]:
import os

In [56]:
clean_file_name = 'cinetel_clean.csv'
clean_file_path = os.path.join(os.getcwd(), clean_file_name)

In [57]:
df1.to_csv(clean_file_path, index=False)
print(f"Il DataFrame è stato esportato con successo in {clean_file_path}.")

Il DataFrame è stato esportato con successo in e:\data_analysis_python\cinetel\cinetel_clean.csv.


---

## Data Exploration

In [58]:
import seaborn as sns
import matplotlib.pyplot as plt

In [59]:
pd.set_option('display.max_rows', 8)

In [60]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 240 entries, 0 to 262
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   daily_rank            240 non-null    int32         
 1   title                 240 non-null    object        
 2   first_screening_date  238 non-null    object        
 3   nation                240 non-null    object        
 4   distribution          240 non-null    object        
 5   daily_takings         240 non-null    float64       
 6   daily_attendance      240 non-null    int32         
 7   total_takings         240 non-null    float64       
 8   total_attendance      240 non-null    int32         
 9   date_pulled           240 non-null    object        
 10  screening_days        240 non-null    int32         
 11  date                  240 non-null    datetime64[ns]
 12  day_of_week           240 non-null    object        
 13  avg_ticket_price      240

In [61]:
df1.describe()

,daily_rank,daily_takings,daily_attendance,total_takings,total_attendance,screening_days,date,avg_ticket_price
count,240.00,240.00,240.00,240.00,240.00,240.00,240,240.00
mean,5.50,153246.73,21677.00,3873959.65,554281.31,25.91,2023-11-27 12:00:00,6.95
min,1.00,8904.68,1216.00,11652.78,1470.00,1.00,2023-11-16 00:00:00,4.22
25%,3.00,24410.58,3943.50,236887.60,33248.75,18.00,2023-11-21 18:00:00,6.30
50%,5.50,54258.99,7533.00,961195.05,131869.50,25.00,2023-11-27 12:00:00,6.91
75%,8.00,183312.94,26275.75,3949289.46,528478.75,33.00,2023-12-03 06:00:00,7.44
max,10.00,1544231.00,211764.00,29167413.88,4266561.00,95.00,2023-12-09 00:00:00,10.94
std,2.88,233320.23,32396.88,6921300.64,1008647.24,15.77,NaN,1.00


In [62]:
df1.isnull().sum()

daily_rank              0
title                   0
first_screening_date    2
nation                  0
                       ..
screening_days          0
date                    0
day_of_week             0
avg_ticket_price        0
Length: 14, dtype: int64

In [63]:
df1.nunique()

daily_rank               10
title                    31
first_screening_date     18
nation                    6
                       ... 
screening_days           19
date                     24
day_of_week               7
avg_ticket_price        240
Length: 14, dtype: int64

In [64]:
df1.corr(numeric_only=True)

,daily_rank,daily_takings,daily_attendance,total_takings,total_attendance,screening_days,avg_ticket_price
daily_rank,1.00,-0.63,-0.65,-0.55,-0.54,-0.10,-0.06
daily_takings,-0.63,1.00,0.99,0.59,0.58,0.19,0.13
daily_attendance,-0.65,0.99,1.00,0.63,0.63,0.22,0.08
total_takings,-0.55,0.59,0.63,1.00,1.00,0.52,-0.13
total_attendance,-0.54,0.58,0.63,1.00,1.00,0.53,-0.14
screening_days,-0.10,0.19,0.22,0.52,0.53,1.00,-0.38
avg_ticket_price,-0.06,0.13,0.08,-0.13,-0.14,-0.38,1.00


In [65]:
df1.groupby('title').mean(numeric_only=True).sort_values(by = 'total_attendance', ascending= False)

,daily_rank,daily_takings,daily_attendance,total_takings,total_attendance,screening_days,avg_ticket_price
title,,,,,,,
C'E' Ancora Domani,1.08,590533.87,87430.33,23549182.19,3429596.21,46.00,6.58
Io Capitano,8.00,16027.86,3670.00,4226997.42,735242.60,95.00,4.38
Me Contro Te Il Film – Vacanze In Transilvania,10.00,37520.80,5383.00,4551425.30,674907.00,53.00,6.97
Killers Of The Flower Moon,9.67,23930.22,3229.33,5007558.46,673423.33,53.00,7.20
...,...,...,...,...,...,...,...
Picasso. Un Ribelle A Parigi. Storia Di Una Vita E Di Un Museo,5.00,27310.89,3306.33,51827.01,6312.00,14.00,8.24
I Goonies 4K - Wb 100Th Anniversary,7.00,20816.21,2819.00,34612.42,4659.00,7.00,7.38
Ferrari,9.00,28661.60,3288.00,30061.60,3470.00,4.00,8.72
Casanova Operapop - Il Film,9.00,12093.47,1256.33,25578.93,2715.33,14.00,9.62


In [66]:
df2 = df1.groupby('title').mean(numeric_only=True).sort_values(by = 'total_attendance', ascending= False)
df2

,daily_rank,daily_takings,daily_attendance,total_takings,total_attendance,screening_days,avg_ticket_price
title,,,,,,,
C'E' Ancora Domani,1.08,590533.87,87430.33,23549182.19,3429596.21,46.00,6.58
Io Capitano,8.00,16027.86,3670.00,4226997.42,735242.60,95.00,4.38
Me Contro Te Il Film – Vacanze In Transilvania,10.00,37520.80,5383.00,4551425.30,674907.00,53.00,6.97
Killers Of The Flower Moon,9.67,23930.22,3229.33,5007558.46,673423.33,53.00,7.20
...,...,...,...,...,...,...,...
Picasso. Un Ribelle A Parigi. Storia Di Una Vita E Di Un Museo,5.00,27310.89,3306.33,51827.01,6312.00,14.00,8.24
I Goonies 4K - Wb 100Th Anniversary,7.00,20816.21,2819.00,34612.42,4659.00,7.00,7.38
Ferrari,9.00,28661.60,3288.00,30061.60,3470.00,4.00,8.72
Casanova Operapop - Il Film,9.00,12093.47,1256.33,25578.93,2715.33,14.00,9.62
